In [1]:
import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from args_reader import read_args
from scicrunch_client_fast import scicrunch
import sys
import time
from interlex_sql import interlex_sql
import re

In [3]:
args = read_args(keys='../keys.txt', production='-p')
sql = interlex_sql(engine_key=args['engine_key'])
sci = scicrunch(key=args['api_key'], base_path=args['base_path'])

In [3]:
relationships_df = sql.get_relationship()

In [4]:
relationships_df

,term1,relationship_id,term2
0,ilx_0100001,ilx_0112784,ilx_0111840
1,ilx_0100002,ilx_0112784,ilx_0111840
2,ilx_0100021,ilx_0112784,ilx_0103720
3,ilx_0100024,ilx_0112784,ilx_0103576
4,ilx_0100026,ilx_0112796,ilx_0106458
5,ilx_0100031,ilx_0112784,ilx_0111264
6,ilx_0100032,ilx_0112784,ilx_0107547
7,ilx_0100032,ilx_0112785,ilx_0110555
8,ilx_0100032,ilx_0112796,ilx_0110557
9,ilx_0100033,ilx_0112784,ilx_0100032


In [ ]:
existing_ids_df = sql.get_existing_ids()

In [6]:
existing_ids_df[existing_ids_df.id == 1]

,id,tid,iri,preferred,ilx,type,label,definition
28149,1,1,http://neurolex.org/wiki/nifext_5214,1,ilx_0100000,term,(R)N6-phenylisopropyladenosine,
28150,1,1,http://uri.neuinfo.org/nif/nifstd/nifext_5214,0,ilx_0100000,term,(R)N6-phenylisopropyladenosine,
28151,1,1,http://uri.interlex.org/base/ilx_0100000,0,ilx_0100000,term,(R)N6-phenylisopropyladenosine,


In [8]:
data =  '''
        SELECT t.id, tei.tid, tei.iri, tei.preferred, t.ilx, t.type, t.label, t.definition
        FROM terms AS t
        JOIN term_existing_ids AS tei ON t.id=tei.tid
        WHERE t.type != 'cde'
        '''
engine = create_engine(args['engine_key'])
df = pd.read_sql(data, engine)

In [9]:
df.loc[df.iri == 'http://neurolex.org/wiki/nif-0000-03485']

,id,tid,iri,preferred,ilx,type,label,definition
19838,7431,7431,http://neurolex.org/wiki/nif-0000-03485,0,ilx_0107430,term,Netherlands Bioinformatics Centre,A collaborative institute of the bioinformatic...
